In [1]:
# !pip3 install dash
# !pip3 install jupyter-dash

In [2]:
# Import required libraries
from jupyter_dash import JupyterDash
from distutils.log import debug
from msilib.schema import Component
from re import A, X
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\1015894\AppData\Local\Temp/ipykernel_18252/1708724438.py:8: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\1015894\AppData\Local\Temp/ipykernel_18252/1708724438.py:9: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Jupyter proxy configiguration

In [3]:
# JupyterDash.infer_jupyter_proxy_config()

In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
spacex_df['dummy'] = '1'
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category,dummy
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0,1
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0,1
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0,1
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0,1
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0,1


In [5]:
# Create an app layout
app = JupyterDash(__name__)
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown( id='site-dropdown',
                                              options=[
                                                        {'label': 'All Sites', 'value': 'ALL'},
                                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                    ],
                                                value='ALL',
                                                placeholder="Select a Launch Site here",
                                                searchable=True
                                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                    2500: '2500',
                                                    5000: '5000',
                                                    7500: '7500',
                                                    10000:'10000'},
                                                value=[0, 10000]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown',component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        filtered_df1 = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig1 = px.pie(filtered_df1, values='dummy', 
        names='class', 
        title= f'Total Success Launches for Site {entered_site}')    
        return fig1
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site,payload):
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)']>payload[0]) & (spacex_df['Payload Mass (kg)']<=payload[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', 
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        filtered_df1 = spacex_df[(spacex_df['Payload Mass (kg)']>payload[0]) & (spacex_df['Payload Mass (kg)']<=payload[1])]
        filtered_df1 = filtered_df1[filtered_df1['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df1, x='Payload Mass (kg)', y='class', color='Booster Version Category', 
        title= f'Correlation between Payload and Success for {entered_site}')    
        return fig

# Run the app
app.run_server(mode='inline',port=9050)